# Western Ghats LULC Analysis using Dynamic World

This notebook analyzes land use and land cover changes in the Western Ghats biodiversity hotspot using Google Earth Engine's Dynamic World dataset.

## Study Area
Western Ghats, India - UNESCO World Heritage biodiversity hotspot covering approximately 107,000 km².

## Data Sources
- **LULC Data**: Google Earth Engine Dynamic World V1 (2015-present)
- **Study Boundary**: Conservation Biology Institute Western Ghats boundary
- **Satellite Data**: Copernicus Sentinel-2 program

## Analysis Period
2018-2024 (2-year intervals, January dry season)

## Methodology
Refined approach using probability bands with quality filtering to ensure geographic accuracy.

## Import Libraries

In [ ]:
import ee
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from shapely.geometry import box
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Load Study Area

In [ ]:
# Load Western Ghats boundary shapefile
shapefile_path = "data/western_ghats_boundary.shp"
western_ghats = gpd.read_file(shapefile_path)

# Convert to WGS84 for Earth Engine
western_ghats_wgs84 = western_ghats.to_crs('EPSG:4326')

# Display basic information
print(f"Study area: {western_ghats.shape[0]} polygon(s)")
print(f"CRS: {western_ghats.crs}")
area_km2 = western_ghats.to_crs('EPSG:3857').area.sum() / 1e6
print(f"Total area: {area_km2:.0f} km²")

# Visualize study area
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
western_ghats_wgs84.plot(ax=ax, color='darkgreen', alpha=0.7, edgecolor='black')
ax.set_title('Western Ghats Study Area', fontsize=14, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.grid(True, alpha=0.3)
plt.show()

## Initialize Google Earth Engine

In [ ]:
# Authenticate and initialize Google Earth Engine
# Run ee.Authenticate() first if not authenticated
ee.Authenticate()

# Initialize with your project ID
ee.Initialize(project='your-gee-project-id')

# Access Dynamic World dataset
dynamic_world = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

# Define LULC classes
lulc_classes = {
    0: 'Water',
    1: 'Trees', 
    2: 'Grass',
    3: 'Flooded vegetation',
    4: 'Crops',
    5: 'Shrub and scrub',
    6: 'Built',
    7: 'Bare',
    8: 'Snow and ice'
}

print("Google Earth Engine initialized successfully!")
print("Dynamic World dataset loaded")

## Prepare Geometry for Analysis

In [ ]:
# Convert Western Ghats boundary to Earth Engine geometry
def geodataframe_to_ee_geometry(gdf):
    """Convert GeoPandas GeoDataFrame to Earth Engine Geometry"""
    geometry = gdf.geometry.iloc[0]
    if geometry.geom_type == 'Polygon':
        coords = [list(geometry.exterior.coords)]
        ee_geometry = ee.Geometry.Polygon(coords)
    elif geometry.geom_type == 'MultiPolygon':
        polygons = []
        for polygon in geometry.geoms:
            coords = [list(polygon.exterior.coords)]
            polygons.append(coords)
        ee_geometry = ee.Geometry.MultiPolygon(polygons)
    return ee_geometry

# Convert to EE geometry
western_ghats_ee = geodataframe_to_ee_geometry(western_ghats_wgs84)

# Verify area calculation
area_ee = western_ghats_ee.area().getInfo()
area_km2 = area_ee / 1e6
print(f"Study area converted to Earth Engine: {area_km2:.0f} km²")

## LULC Analysis Function

In [ ]:
def calculate_lulc_refined(year, geometry, lulc_classes):
    """
    Calculate LULC using Dynamic World probability bands with quality filtering
    """
    try:
        print(f"Processing January {year}...")
        
        start = f'{year}-01-01'
        end = f'{year}-01-31'
        
        # Get Dynamic World collection for January (dry season)
        dw_collection = dynamic_world.filterDate(start, end).filterBounds(geometry)
        count = dw_collection.size().getInfo()
        
        if count == 0:
            print(f"No images found for January {year}")
            return None
            
        print(f"Found {count} images for January {year}")
        
        # Define probability bands
        prob_bands = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 
                     'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
        
        # Quality filtering function
        def add_quality_score(image):
            # Penalize snow/ice probability (impossible for Western Ghats)
            snow_prob = image.select('snow_and_ice')
            snow_penalty = snow_prob.multiply(-10)
            
            # Quality score based on maximum probability confidence
            max_prob = image.select(prob_bands).reduce(ee.Reducer.max())
            quality = max_prob.add(snow_penalty)
            
            return image.addBands(quality.rename('quality'))
        
        # Create quality-filtered composite
        quality_collection = dw_collection.map(add_quality_score)
        best_composite = quality_collection.qualityMosaic('quality').clip(geometry)
        
        # Create refined label from probability bands
        def create_refined_label(image):
            probs = image.select(prob_bands)
            
            # Apply minimum threshold and eliminate snow/ice
            min_threshold = 0.3
            probs_thresh = probs.gte(min_threshold)
            probs_clean = probs.multiply(probs_thresh)
            probs_clean = probs_clean.addBands(ee.Image.constant(0).rename('snow_and_ice'), overwrite=True)
            
            # Get class with maximum probability
            label = probs_clean.toArray().arrayArgmax().arrayGet([0])
            return label.rename('refined_label')
        
        refined_label = create_refined_label(best_composite)
        
        # Calculate areas for each class
        stats = {'year': year}
        total_area = 0
        
        for class_id, class_name in lulc_classes.items():
            if class_name == 'Snow and ice':
                stats[class_name] = 0.0
                continue
                
            class_mask = refined_label.eq(class_id)
            area = class_mask.multiply(ee.Image.pixelArea()).reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=10,
                maxPixels=1e9,
                bestEffort=True
            ).getInfo()
            
            area_m2 = area.get('refined_label', 0)
            area_km2 = area_m2 / 1e6
            
            stats[class_name] = area_km2
            total_area += area_km2
        
        # Add percentages
        for class_name in lulc_classes.values():
            if class_name != 'Snow and ice':
                percentage = (stats[class_name] / total_area * 100) if total_area > 0 else 0
                stats[f'{class_name}_percent'] = percentage
        
        stats['total_area'] = total_area
        print(f"January {year} completed - Total: {total_area:.0f} km²")
        
        return stats
        
    except Exception as e:
        print(f"Error processing January {year}: {e}")
        return None

print("LULC analysis function defined")

## Run Analysis

In [ ]:
# Run LULC analysis for strategic years
analysis_years = [2018, 2020, 2022, 2024]
results = []

print("Starting LULC analysis...")
print("This will take 30-60 minutes for the full area")

for year in analysis_years:
    stats = calculate_lulc_refined(year, western_ghats_ee, lulc_classes)
    if stats:
        results.append(stats)
        print(f"✓ {year} completed")
    else:
        print(f"✗ {year} failed")

if results:
    # Convert to DataFrame
    lulc_df = pd.DataFrame(results)
    
    print(f"\nAnalysis completed for {len(results)} years")
    print("Key results:")
    key_cols = ['year', 'Trees', 'Built', 'Crops', 'total_area']
    print(lulc_df[key_cols].round(1))
    
    # Save results
    lulc_df.to_csv('outputs/western_ghats_lulc_results.csv', index=False)
    print("\nResults saved to outputs/western_ghats_lulc_results.csv")
else:
    print("No results obtained. Check Earth Engine authentication.")

## Visualize Results

In [ ]:
# Create visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. LULC area changes
main_classes = ['Trees', 'Built', 'Crops', 'Grass', 'Shrub and scrub', 'Water']
colors = {'Trees': 'darkgreen', 'Built': 'red', 'Crops': 'gold', 
          'Grass': 'lightgreen', 'Shrub and scrub': 'brown', 'Water': 'blue'}

for class_name in main_classes:
    ax1.plot(lulc_df['year'], lulc_df[class_name], 
             marker='o', linewidth=2, label=class_name, color=colors[class_name])

ax1.set_title('Land Cover Changes (2018-2024)', fontweight='bold')
ax1.set_xlabel('Year')
ax1.set_ylabel('Area (km²)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Built-up area focus
ax2.plot(lulc_df['year'], lulc_df['Built'], 
         marker='s', linewidth=3, markersize=8, color='red')
ax2.fill_between(lulc_df['year'], lulc_df['Built'], alpha=0.3, color='red')

ax2.set_title('Built-up Area Expansion', fontweight='bold')
ax2.set_xlabel('Year')
ax2.set_ylabel('Built-up Area (km²)')
ax2.grid(True, alpha=0.3)

# 3. Percentage composition
ax3.plot(lulc_df['year'], lulc_df['Trees_percent'], 
         marker='o', linewidth=3, color='darkgreen', label='Forest')
ax3.plot(lulc_df['year'], lulc_df['Built_percent'], 
         marker='s', linewidth=3, color='red', label='Built-up')

ax3.set_title('Forest vs Built-up Trends', fontweight='bold')
ax3.set_xlabel('Year')
ax3.set_ylabel('Percentage (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Annual growth rates
if len(lulc_df) > 1:
    growth_rates = lulc_df['Built'].pct_change() * 100
    ax4.bar(lulc_df['year'][1:], growth_rates[1:], color='orange', alpha=0.8)
    
    ax4.set_title('Built-up Area Growth Rates', fontweight='bold')
    ax4.set_xlabel('Year')
    ax4.set_ylabel('Growth Rate (%)')
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/lulc_analysis_charts.png', dpi=300, bbox_inches='tight')
plt.show()

# Print summary statistics
print("\nSummary Statistics:")
print(f"Study period: {lulc_df['year'].min()}-{lulc_df['year'].max()}")
print(f"Total area: {lulc_df['total_area'].mean():.0f} km²")

initial_built = lulc_df['Built'].iloc[0]
final_built = lulc_df['Built'].iloc[-1]
built_growth = ((final_built / initial_built) - 1) * 100

print(f"Built-up area 2018: {initial_built:.0f} km²")
print(f"Built-up area 2024: {final_built:.0f} km²")
print(f"Total growth: {built_growth:.1f}%")

print(f"Forest coverage 2018: {lulc_df['Trees_percent'].iloc[0]:.1f}%")
print(f"Forest coverage 2024: {lulc_df['Trees_percent'].iloc[-1]:.1f}%")